In [1]:
import requests

url = "https://api.os.uk/downloads/v1/products/CodePointOpen/downloads?area=GB&format=CSV&redirect"  # direct link
out_path = "codepo_gb.zip"

resp = requests.get(url, timeout=30)
resp.raise_for_status()  # raise if 4xx/5xx
with open(out_path, "wb") as f:
    f.write(resp.content)

print(f"Saved to {out_path}")


Saved to codepo_gb.zip


In [2]:
import os
import pandas as pd
import zipfile

curDir = os.getcwd()
zf = zipfile.ZipFile(curDir + '/codepo_gb.zip')
text_files = zf.infolist()

print ("Uncompressing and reading data... ")
#print(text_files)


Uncompressing and reading data... 


In [34]:
list_ = []
alist_ = []
clist_ = []
nlist_ = []
codelist1 = pd.DataFrame(columns=['AreaDescription', 'AreaCode','AreaTypeCode', 'AreaType'])
nhs1 = pd.DataFrame(columns=['AreaCode','AreaDescription','AreaTypeCode', 'AreaType'])
for text_file in text_files:

    #Define columns of the csv reading
    colhd = ['Postcode','Positional_quality_indicator','Eastings','Northings','Country_code','NHS_regional_HA_code','NHS_HA_code','Admin_county_code','Admin_district_code','Admin_ward_code']

    # Session of main csv files
    # print(text_file.filename)
    if text_file.filename.endswith("csv"):
        if text_file.filename.startswith("Data/CSV/"):
            df = pd.read_csv(zf.open(text_file.filename), names=colhd)
            list_.append(df)

    # Session of info Excel (Area Code Lib)
    elif text_file.filename.endswith("xlsx"):
        if text_file.filename.startswith("Doc/Codelist"):
            with zf.open(text_file.filename) as f:
                #area = pd.read_excel(text_file.filename,"AREA_CODES", dtype={"AreaTypeCode": str, "AreaType": str})
                area = pd.read_excel(f,"AREA_CODES", header=None, names=['AreaTypeCode', 'AreaType'])
            alist_.append(area)
            #print(area)

            x = 0
            for i in area['AreaTypeCode']:
                #print(i)
                with zf.open(text_file.filename) as f1:
                    codelist = pd.read_excel(f1, i, header=None, names=['AreaDescription', 'AreaCode'])
                    codelist["AreaTypeCode"] = i
                    codelist["AreaType"] = area["AreaType"].iloc[x]
                    
                    clist_.append(codelist)
                    codelist.head()
        
            #pointer point to next
            x = x + 1
        codelist1 = pd.concat(clist_)

    # Session of NHS area code
    elif text_file.filename.startswith("Doc/NHS_Codelist"):
        with zf.open(text_file.filename) as f:
            nhs = pd.read_excel(f,"English SHA", header=None, names=['AreaCode','AreaDescription'])
            #Adding 2 dummy columns
            nhs["AreaTypeCode"] = 'NaN'
            nhs["AreaType"] = 'NaN'
            nlist_.append(nhs)
        with zf.open(text_file.filename) as f:
            nhs = pd.read_excel(f,"English PanSHA", header=None, names=['AreaCode','AreaDescription'])
            #Adding 2 dummy columns
            nhs["AreaTypeCode"] = 'NaN'
            nhs["AreaType"] = 'NaN'
            nlist_.append(nhs)
        with zf.open(text_file.filename) as f:
            nhs = pd.read_excel(f,"Welsh Local Health Boards", header=None, names=['AreaCode','AreaDescription'])
            #Adding 2 dummy columns
            nhs["AreaTypeCode"] = 'NaN'
            nhs["AreaType"] = 'NaN'
            nlist_.append(nhs)
        with zf.open(text_file.filename) as f:
            nhs = pd.read_excel(f,"Scottish Health Boards", header=None, names=['AreaCode','AreaDescription'])
            #Adding 2 dummy columns
            nhs["AreaTypeCode"] = 'NaN'
            nhs["AreaType"] = 'NaN'
            nlist_.append(nhs)
        with zf.open(text_file.filename) as f:
            nhs = pd.read_excel(f,"NI Health Board", header=None, names=['AreaCode','AreaDescription'])
            #Adding 2 dummy columns
            nhs["AreaTypeCode"] = 'NaN'
            nhs["AreaType"] = 'NaN'
            nlist_.append(nhs)
            
        nhs1 = pd.concat(nlist_)

    # reorganise column as desire order
    desir_order = ["AreaTypeCode","AreaType","AreaDescription","AreaCode"]
    codelist1 = codelist1[desir_order]

    # reorganise column as desire order
    nhs1 = nhs1[desir_order]

df = pd.concat(list_)
#concat 2 pandas data frame into cl (another data frame as final result)
cl0 = pd.concat([codelist1, nhs1], axis=0, ignore_index=True)

cl = (
    cl0
      .groupby(["AreaCode","AreaTypeCode","AreaType"], as_index=False)["AreaDescription"]
      .max()
)

cl = cl[desir_order]
cl.head()

,AreaTypeCode,AreaType,AreaDescription,AreaCode
0,MTW,County,Ainsdale Ward,E05000932
1,MTW,County,Birkdale Ward,E05000933
2,MTW,County,Blundellsands Ward,E05000934
3,MTW,County,Cambridge Ward,E05000935
4,MTW,County,Church Ward,E05000936


In [4]:
cl.head()

,AreaTypeCode,AreaType,AreaDescription,AreaCode
0,MTW,County,Ainsdale Ward,E05000932
1,MTW,County,Birkdale Ward,E05000933
2,MTW,County,Blundellsands Ward,E05000934
3,MTW,County,Cambridge Ward,E05000935
4,MTW,County,Church Ward,E05000936


In [5]:
#check the CodePoint result set length, how many rows in there.
len(df)

1745849

In [6]:
len(cl)

8352

In [7]:
from pathlib import Path  
filepath = Path('data/codepoint_combine.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath)  

parpath = Path('data/codepoint_combine.parquet')
parpath.parent.mkdir(parents=True, exist_ok=True)
df.to_parquet(parpath)

In [8]:
df.head()

,Postcode,Positional_quality_indicator,Eastings,Northings,Country_code,NHS_regional_HA_code,NHS_HA_code,Admin_county_code,Admin_district_code,Admin_ward_code
0,AB10 1AB,10,394235,806529,S92000003,NaN,S08000020,NaN,S12000033,S13002842
1,AB10 1AF,10,394235,806529,S92000003,NaN,S08000020,NaN,S12000033,S13002842
2,AB10 1AG,10,394230,806469,S92000003,NaN,S08000020,NaN,S12000033,S13002842
3,AB10 1AH,10,394235,806529,S92000003,NaN,S08000020,NaN,S12000033,S13002842
4,AB10 1AL,10,394296,806581,S92000003,NaN,S08000020,NaN,S12000033,S13002842


In [9]:
df.dtypes

Postcode                        object
Positional_quality_indicator     int64
Eastings                         int64
Northings                        int64
Country_code                    object
NHS_regional_HA_code            object
NHS_HA_code                     object
Admin_county_code               object
Admin_district_code             object
Admin_ward_code                 object
dtype: object

In [9]:
df1 = df.astype({'Postcode':'object','Positional_quality_indicator':'object','Eastings':'object','Northings':'object','Country_code':'object','NHS_regional_HA_code':'object','NHS_HA_code':'object','Admin_county_code':'object','Admin_district_code':'object','Admin_ward_code':'object'})

In [10]:
df1.dtypes

Postcode                        object
Positional_quality_indicator    object
Eastings                        object
Northings                       object
Country_code                    object
NHS_regional_HA_code            object
NHS_HA_code                     object
Admin_county_code               object
Admin_district_code             object
Admin_ward_code                 object
dtype: object

In [11]:
# Export reseult set to Landing.CodePointJan2026
# Import SQL library : sqlalchemy

from sqlalchemy import create_engine
from sqlalchemy.types import Integer, String, Numeric, Float

server = 'cic-dw02'
database = 'cicedm01_dev'
engine = create_engine(
    'mssql+pyodbc://'+server+'/'+database+'?driver=ODBC+Driver+17+for+SQL+Server',fast_executemany=True
    # Or ODBC Driver 18
)

dtype_map = {
    "Postcode": String(10), 
    "Positional_quality_indicator": String(10),
    "Eastings": String(10),
    "Northings": String(10),
    "Country_code": String(10),
    "NHS_regional_HA_code": String(10),
    "NHS_HA_code": String(10),
    "Admin_county_code": String(10),
    "Admin_district_code": String(10),
    "Admin_ward_code": String(10),
}

df1.to_sql(
    name="CodePointJan2026",
    con=engine,
    schema="Landing",
    if_exists="replace",
    index=False,
    chunksize = 5000,
    dtype=dtype_map
    #method="multi"   # batch insert
)


-350

In [12]:
# Close connection with SQL Server
engine.dispose()

In [13]:
cl.dtypes

AreaTypeCode       object
AreaType           object
AreaDescription    object
AreaCode           object
dtype: object

In [15]:
## Export the Adress postcode lib to SQL Server

server = 'cic-dw02'
database = 'cicedm01_dev'
engine = create_engine(
    'mssql+pyodbc://'+server+'/'+database+'?driver=ODBC+Driver+17+for+SQL+Server',fast_executemany=True
    # Or ODBC Driver 18
)

dtype_map = {
    "AreaTypeCode": String(5), 
    "AreaType": String(100),
    "AreaDescription": String(500),
    "AreaCode": String(50)
}

cl.to_sql(
    name="CodePointAreaDescriptions",
    con=engine,
    schema="Landing",
    if_exists="replace",
    index=False,
    chunksize = 5000,
    dtype=dtype_map
    #method="multi"   # batch insert
)

-2

In [16]:
engine.dispose()

In [39]:
# Simulate SQL script for joining tables to generate result Base.CodePoint
import datetime
# Prepare reference dataframe (only key column and reference description column)
cl1 = (
    cl
      .groupby(["AreaCode", "AreaDescription"], as_index=False)["AreaDescription"]
      .max()
)
cl1.head()

# Join above reference data frame to cp1 data frame (left join: how="left")
cp1 = df1.merge(cl1, how="left", left_on=["Admin_ward_code"], right_on=["AreaCode"]).drop(columns=["AreaCode"]).rename(columns={"AreaDescription": "Admin_ward_name"})\
.merge(cl1, how="left", left_on=["Admin_district_code"], right_on=["AreaCode"]).drop(columns=["AreaCode"]).rename(columns={"AreaDescription": "Admin_district_name"})\
.merge(cl1, how="left", left_on=["Admin_county_code"], right_on=["AreaCode"]).drop(columns=["AreaCode"]).rename(columns={"AreaDescription": "Admin_county_name"})\
.merge(cl1, how="left", left_on=["NHS_HA_code"], right_on=["AreaCode"]).drop(columns=["AreaCode"]).rename(columns={"AreaDescription": "NHS_HA_name"})\
.merge(cl1, how="left", left_on=["NHS_regional_HA_code"], right_on=["AreaCode"]).drop(columns=["AreaCode"]).rename(columns={"AreaDescription": "NHS_regional_HA_name"})\
.merge(cl1, how="left", left_on=["Country_code"], right_on=["AreaCode"]).drop(columns=["AreaCode"]).rename(columns={"AreaDescription": "Country_name"})

# Reorganise columns of result set
desir_order = ["Postcode","Positional_quality_indicator","Eastings","Northings","Country_code","Country_name","NHS_regional_HA_code","NHS_regional_HA_name","NHS_HA_code","NHS_HA_name",\
               "Admin_county_code","Admin_county_name","Admin_district_code","Admin_district_name","Admin_ward_code","Admin_ward_name"]
cp1 = cp1[desir_order]
cp1["RecordCreatedDate"] = datetime.datetime.now()
cp1["RecordLastUpdatedDate"] = datetime.datetime.now()
cp1["SourceFileDate"] = datetime.datetime.now()
# Show result data frame
cp1.head()

,Postcode,Positional_quality_indicator,Eastings,Northings,Country_code,Country_name,NHS_regional_HA_code,NHS_regional_HA_name,NHS_HA_code,NHS_HA_name,Admin_county_code,Admin_county_name,Admin_district_code,Admin_district_name,Admin_ward_code,Admin_ward_name,RecordCreatedDate,RecordLastUpdatedDate,SourceFileDate
0,AB10 1AB,10,394235,806529,S92000003,NaN,NaN,NaN,S08000020,Grampian,NaN,NaN,S12000033,Aberdeen City,S13002842,George St/Harbour Ward,2026-01-30 16:42:05.328917,2026-01-30 16:42:05.331725,2026-01-30 16:42:05.334937
1,AB10 1AF,10,394235,806529,S92000003,NaN,NaN,NaN,S08000020,Grampian,NaN,NaN,S12000033,Aberdeen City,S13002842,George St/Harbour Ward,2026-01-30 16:42:05.328917,2026-01-30 16:42:05.331725,2026-01-30 16:42:05.334937
2,AB10 1AG,10,394230,806469,S92000003,NaN,NaN,NaN,S08000020,Grampian,NaN,NaN,S12000033,Aberdeen City,S13002842,George St/Harbour Ward,2026-01-30 16:42:05.328917,2026-01-30 16:42:05.331725,2026-01-30 16:42:05.334937
3,AB10 1AH,10,394235,806529,S92000003,NaN,NaN,NaN,S08000020,Grampian,NaN,NaN,S12000033,Aberdeen City,S13002842,George St/Harbour Ward,2026-01-30 16:42:05.328917,2026-01-30 16:42:05.331725,2026-01-30 16:42:05.334937
4,AB10 1AL,10,394296,806581,S92000003,NaN,NaN,NaN,S08000020,Grampian,NaN,NaN,S12000033,Aberdeen City,S13002842,George St/Harbour Ward,2026-01-30 16:42:05.328917,2026-01-30 16:42:05.331725,2026-01-30 16:42:05.334937




**Below is demonstrate some query in Pandas**



In [40]:
# Check duplicate record by Postcode
ChkDup = (
    cp1.groupby("Postcode")
      .size()
      .reset_index(name="RecCnt")
)

ChkDup = ChkDup[ChkDup["RecCnt"] > 1]


In [41]:
ChkDup.head()

,Postcode,RecCnt


In [42]:
# Instant check duplicate record by Postcode

(df1.groupby("Postcode").size().reset_index(name="RecCnt").pipe(lambda x: x[x.RecCnt > 1])).head()

,Postcode,RecCnt


In [43]:
(cl.groupby("AreaCode").size().reset_index(name="RecCnt").pipe(lambda x: x[x.RecCnt > 1])).head()

,AreaCode,RecCnt


In [44]:
# Instant query dataframe by filter
cl[cl["AreaCode"] == "E05005782"]

,AreaTypeCode,AreaType,AreaDescription,AreaCode
617,DIW,County,Thorpe St. Andrew South East Ward (DET),E05005782


In [46]:
cp1[cp1["Postcode"] == "CB24 6AB"]

,Postcode,Positional_quality_indicator,Eastings,Northings,Country_code,Country_name,NHS_regional_HA_code,NHS_regional_HA_name,NHS_HA_code,NHS_HA_name,Admin_county_code,Admin_county_name,Admin_district_code,Admin_district_name,Admin_ward_code,Admin_ward_name,RecordCreatedDate,RecordLastUpdatedDate,SourceFileDate
213976,CB24 6AB,10,547953,262853,E92000001,NaN,E19000001,"North, Midlands and the East Programme for IT ...",E18000006,East of England,E10000003,Cambridgeshire County,E07000012,South Cambridgeshire District,E05011301,Milton & Waterbeach Ward (DET),2026-01-30 16:42:05.328917,2026-01-30 16:42:05.331725,2026-01-30 16:42:05.334937


In [51]:
# Filter query as SQL like. str.contains ", na=False" is handling null records

cp1[cp1["Admin_county_name"].str.contains("Cambridge", na=False)]

,Postcode,Positional_quality_indicator,Eastings,Northings,Country_code,Country_name,NHS_regional_HA_code,NHS_regional_HA_name,NHS_HA_code,NHS_HA_name,Admin_county_code,Admin_county_name,Admin_district_code,Admin_district_name,Admin_ward_code,Admin_ward_name,RecordCreatedDate,RecordLastUpdatedDate,SourceFileDate
208762,CB1 0AG,10,546182,257043,E92000001,NaN,E19000001,"North, Midlands and the East Programme for IT ...",E18000006,East of England,E10000003,Cambridgeshire County,E07000008,Cambridge District (B),E05013054,Coleridge Ward,2026-01-30 16:42:05.328917,2026-01-30 16:42:05.331725,2026-01-30 16:42:05.334937
208763,CB1 0AH,10,546182,257043,E92000001,NaN,E19000001,"North, Midlands and the East Programme for IT ...",E18000006,East of England,E10000003,Cambridgeshire County,E07000008,Cambridge District (B),E05013054,Coleridge Ward,2026-01-30 16:42:05.328917,2026-01-30 16:42:05.331725,2026-01-30 16:42:05.334937
208764,CB1 0AN,10,546182,257043,E92000001,NaN,E19000001,"North, Midlands and the East Programme for IT ...",E18000006,East of England,E10000003,Cambridgeshire County,E07000008,Cambridge District (B),E05013054,Coleridge Ward,2026-01-30 16:42:05.328917,2026-01-30 16:42:05.331725,2026-01-30 16:42:05.334937
208765,CB1 0AS,10,546182,257043,E92000001,NaN,E19000001,"North, Midlands and the East Programme for IT ...",E18000006,East of England,E10000003,Cambridgeshire County,E07000008,Cambridge District (B),E05013054,Coleridge Ward,2026-01-30 16:42:05.328917,2026-01-30 16:42:05.331725,2026-01-30 16:42:05.334937
208766,CB1 0AT,10,546182,257043,E92000001,NaN,E19000001,"North, Midlands and the East Programme for IT ...",E18000006,East of England,E10000003,Cambridgeshire County,E07000008,Cambridge District (B),E05013054,Coleridge Ward,2026-01-30 16:42:05.328917,2026-01-30 16:42:05.331725,2026-01-30 16:42:05.334937
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1375521,SG8 8SW,10,542289,238626,E92000001,NaN,E19000001,"North, Midlands and the East Programme for IT ...",E18000006,East of England,E10000003,Cambridgeshire County,E07000012,South Cambridgeshire District,E05011292,Foxton Ward,2026-01-30 16:42:05.328917,2026-01-30 16:42:05.331725,2026-01-30 16:42:05.334937
1375522,SG8 8TH,10,543218,240068,E92000001,NaN,E19000001,"North, Midlands and the East Programme for IT ...",E18000006,East of England,E10000003,Cambridgeshire County,E07000012,South Cambridgeshire District,E05011292,Foxton Ward,2026-01-30 16:42:05.328917,2026-01-30 16:42:05.331725,2026-01-30 16:42:05.334937
1375665,SG8 9NJ,10,532542,240649,E92000001,NaN,E19000001,"North, Midlands and the East Programme for IT ...",E18000006,East of England,E10000003,Cambridgeshire County,E07000012,South Cambridgeshire District,E05011285,Bassingbourn Ward,2026-01-30 16:42:05.328917,2026-01-30 16:42:05.331725,2026-01-30 16:42:05.334937
1375669,SG8 9NR,10,530767,240507,E92000001,NaN,E19000001,"North, Midlands and the East Programme for IT ...",E18000006,East of England,E10000003,Cambridgeshire County,E07000012,South Cambridgeshire District,E05011306,The Mordens Ward,2026-01-30 16:42:05.328917,2026-01-30 16:42:05.331725,2026-01-30 16:42:05.334937


In [53]:
cl.head(200)

,AreaTypeCode,AreaType,AreaDescription,AreaCode
0,MTW,County,Ainsdale Ward,E05000932
1,MTW,County,Birkdale Ward,E05000933
2,MTW,County,Blundellsands Ward,E05000934
3,MTW,County,Cambridge Ward,E05000935
4,MTW,County,Church Ward,E05000936
...,...,...,...,...
195,MTW,County,Darlaston South Ward,E05001309
196,MTW,County,Paddock Ward,E05001310
197,MTW,County,Palfrey Ward,E05001311
198,MTW,County,Pelsall Ward,E05001312
